In [43]:
import yfinance as yf
from pypfopt import expected_returns, EfficientSemivariance, EfficientFrontier, plotting
from pypfopt.risk_models import risk_matrix

OSError: 'seaborn-deep' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

In [ ]:
data = yf.download(['NVDA', 'AMD'], start='2020-01-01', end='2020-06-01')['Close']
data

In [ ]:
mu = expected_returns.mean_historical_return(data)
historicalReturns = expected_returns.returns_from_prices(data)

In [38]:
mu

AMD     0.253384
NVDA    1.633329
dtype: float64

In [39]:
historicalReturns

,AMD,NVDA
Date,,
2020-01-03,-0.010183,-0.016006
2020-01-06,-0.004321,0.004194
2020-01-07,-0.002893,0.012107
2020-01-08,-0.008705,0.001876
2020-01-09,0.023834,0.010983
...,...,...
2020-05-22,0.009515,0.028603
2020-05-26,-0.035889,-0.034178
2020-05-27,-0.008460,-0.022081


In [40]:
es = EfficientSemivariance(mu, historicalReturns)
es.efficient_return(0.20)

OrderedDict([('AMD', 0.5544825089837042), ('NVDA', 0.4455174910063497)])

In [41]:
weights = es.clean_weights()
weights

OrderedDict([('AMD', 0.55448), ('NVDA', 0.44552)])

In [42]:
es.portfolio_performance(verbose=True)

Expected annual return: 86.8%
Annual semi-deviation: 48.4%
Sortino Ratio: 1.75


(0.8681738558311758, 0.4842280011241939, 1.7516001839258315)

In [36]:
S = risk_matrix(data)
ef = EfficientFrontier(mu, S)
ef